# 练习集 2

### 判断题

**问题1.** 假设本周的太阳能预测总量超过了总需求。在这种情况下，可以保证不会从电网购买能源。
- 正确
- `错误`

### 建模和编程

In [ ]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB

batteries = ["Battery0", "Battery1"]
time_periods = range(180)
capacity = {"Battery0": 60, "Battery1": 80} 
p_loss = {"Battery0": 0.95, "Battery1": 0.9} 
initial = {"Battery0": 20, "Battery1": 30} 

m = gp.Model() 

# flow_in = m.addVars(batteries, time_periods, name="flow_in") 
# flow_out = m.addVars(batteries, time_periods, name="flow_out")
# grid = m.addVars(time_periods, name="grid")
# state = m.addVars(batteries, time_periods, name="state") 
# gen = m.addVars(time_periods, name="gen")

欢迎使用上面被注释掉的代码进行复制粘贴。

**问题2.** 使用`addVars()`命令编写一行代码，为能源存储问题设置目标系数（最小化购买的电网电力）。通过将模型写入名为`model.lp`的文件来检查你的代码。

In [ ]:
#### gurobipy代码
grid = m.addVars(time_periods, obj = 1, name="grid")
m.write('model.lp')

**问题3.** [放电深度](https://en.wikipedia.org/wiki/Depth_of_discharge)指的是电池电量低于其容量的百分比。为了帮助维护电池，施加了一个限制，即**两个电池合计**的放电深度不能超过**总容量**的70%。建立模型约束条件（用代数方式编写不等式）并编写gurobipy代码。

**提示：** 如果出错，请确保你的生成器表达式在括号内，并考虑上方几个单元格中提到的初始状态。

公式：
$$
\begin{align*}
\sum_bs_{b,t} \ge (1-0.7)*\sum_bc_b, \quad \forall t \in T
\end{align*}
$$

In [ ]:
#### gurobipy代码
state = m.addVars(batteries, time_periods, name="state") 
depth_of_discharge = m.addConstrs((gp.quicksum(state[b,t] for b in batteries) >= 0.3*sum(capacity[b] for b in batteries) for t in time_periods), name = 'depth_restriction')

**问题4.** 在每天开始时（第一天除外），要求每个电池至少充满40%。每天开始的时间段为30、60、90、120和150。制定这些约束条件并编写gurobipy代码。不必担心太阳能预测和**问题3**中的约束条件。

公式：
$$
\begin{align*}
s_{b,t} \ge 0.4*c_b, \quad \forall t \in [30,60,90,120,150]
\end{align*}
$$

In [ ]:
#### gurobipy代码
m = gp.Model() # 再次定义模型以确保它被重置
state = m.addVars(batteries, time_periods, name="state") 
start_of_day = m.addConstrs((state[b,t] >= 0.4*capacity[b] for b in batteries for t in [30,60,90,120,150]), name = 'start_of_day') #in time_periods if t

### 使用二元变量建模

你有五个灯泡。设$y_i = 1$表示灯泡$i$是开启的，$y_i = 0$表示灯泡是关闭的，对于$\space i \in \{1,2,3,4,5\}$。使用二元变量和$\le,\space\ge,\space=$来建模以下陈述。所有问题彼此独立。

**问题4.** 最多只能开启三个灯泡。
$$
\begin{align*}
\sum_i y_i \le 3
\end{align*}
$$

**问题5.** 灯泡2或灯泡3必须开启。
$$
\begin{align*}
y_2 + y_3 \ge 1
\end{align*}
$$

**问题6.** 如果灯泡1开启，那么灯泡5也开启。
$$
\begin{align*} 
y_1 \le y_5
\end{align*}
$$

**问题7.** 如果灯泡1关闭，那么灯泡3开启。
$$
\begin{align*} 
1-y_1 \le y_3
\end{align*}
$$

**问题8.** 如果灯泡1关闭，那么灯泡2关闭。
$$
\begin{align*} 
1-y_1 &\le 1-y_2 \\
y_1 &\ge y_2
\end{align*}
$$

**问题9.** 灯泡2和灯泡4处于相反的开/关状态。
$$
\begin{align*} 
y_2 =& 1-y_4 \\
y_2 + y_4 =& 1
\end{align*}
$$

**问题10.** 如果灯泡1到4中的任何一个开启，那么灯泡5也开启。
$$
\begin{align*}
y_1 + y_2 + y_3 + y_4 \le 4y_5
\end{align*}
$$